In [ ]:
# Importing libraries
import cv2 as cv
import pandas as pd
import numpy as np
import os
import shutil

# Accessing to the data through Google Drive in colab 
spectrogram_path = "drive/MyDrive/spectrograms/"
processed_path = "drive/MyDrive/spectrograms_processed"

In [ ]:
# Spectrogram Image Processing
def spectrogram_process(path):
  '''
  This function gets the image through the given path variable,
  turns the image into grayscale, scales the image down and normalizes.
  Returns numpy array.

  '''
  img = cv.imread(path, cv.IMREAD_UNCHANGED)
  grayscaled_img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

  #scale = 0.5 # original size * scale
  new_width = 32
  new_height = 32

  dimensions = (new_width,new_height)
  resized = cv.resize(grayscaled_img,dimensions, interpolation = cv.INTER_AREA)

  normalizedImg = np.zeros((800, 800))
  final_img = cv.normalize(resized,  normalizedImg, 0, 255, cv.NORM_MINMAX)
  img_expanded = final_img[:, :, np.newaxis]
  return img_expanded

In [ ]:
# Getting images from Google Drive, processing them and saving them.
# After this process, put images into a folder named "spectrograms_processed" MANUALLY!!!.

df = pd.DataFrame(columns=['image_name', 'label'])
for fold in range(0,10):
  path = spectrogram_path + str(fold)
  save_path = processed_path + str(fold)
  for f in os.listdir(path):
    df = df.append({'image_name': f, 'label': fold}, ignore_index=True)
    cv.imwrite(f,spectrogram_process(path + "/" + f))

(32, 32, 1)
a (374, 500, 3)


In [ ]:
def get_label(file_name):
  '''
  This function gets the file name and returns the label of the file from its name.
  '''
  return file_name.split('.png')[0].split('-')[1]

In [ ]:
# Creating a dataframe with the processed images and their labels.
i = 0
for f in os.listdir("spectrogram_processed"):
    i +=1
    print("Label:",get_label(f))
    path = 'spectrogram_processed'
    if not os.path.exists(os.path.join(path,get_label(f))):      
        os.makedirs(os.path.join(path,get_label(f)))  
    shutil.move(os.path.join(path,f),os.path.join(path, get_label(f),get_label(f) + "_" + str(i)+".png"))

In [ ]:
def train_test_split(root_dir,processed_dir,classes_dir,val_ratio=0.20,test_ratio=0.20):
    '''
    This function gets the root directory of the images, the processed data directory and the classes.
    It splits the spectrograms into train, test and validation sets.
    '''

    for cls in classes_dir:
        print("- Class Name " + cls)
        src = processed_dir +"//" + cls  # Folder to copy images from

        allFileNames = os.listdir(src)
        np.random.shuffle(allFileNames)
        train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                                  [int(len(allFileNames) * (1 - (val_ratio + test_ratio))),
                                                                   int(len(allFileNames) * (1 - val_ratio)),
                                                                   ])

        train_FileNames = [src + '//' + name for name in train_FileNames.tolist()]
        val_FileNames = [src + '//' + name for name in val_FileNames.tolist()]
        test_FileNames = [src + '//' + name for name in test_FileNames.tolist()]

        print('Total images: '+ str(len(allFileNames)))
        print('Training: '+ str(len(train_FileNames)))
        print('Validation: '+  str(len(val_FileNames)))
        print('Testing: '+ str(len(test_FileNames)))

        # # Creating Train / Val / Test folders (One time use)
        os.makedirs(root_dir + '/train//' + cls)
        os.makedirs(root_dir + '/val//' + cls)
        os.makedirs(root_dir + '/test//' + cls)

        # Copy-pasting images
        for name in train_FileNames:
            shutil.copy(name, root_dir + '/train//' + cls)
        for name in val_FileNames:
            shutil.copy(name, root_dir + '/val//' + cls)
        for name in test_FileNames:
            shutil.copy(name, root_dir + '/test//' + cls)

train_test_split("spectrogram_data","spectrogram_processed",["0","1","2","3","4","5","6","7","8","9"],val_ratio=0.0,test_ratio=0.2)